In [ ]:
import pydicom
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
dcmImage = pydicom.dcmread("trial.dcm")
dcm_scan = dcmImage.pixel_array
print(np.shape(dcm_scan))

In [6]:
from scipy.io import loadmat
from matplotlib import pyplot as plt
import os.path
import numpy as np

In [4]:
matFile = loadmat(os.path.join('data', 'fractals_192.mat'))
matImage = matFile['X']
ImageTICI = matFile['TICI_report']

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        image = loadmat(os.path.join(folder, filename))
        images.append(image)
    return images

prelim_images = load_images_from_folder('data')

In [20]:
np.append([[1, 2, 3], [4, 5, 6]], [[7, 8, 9]], axis=0)
print(np.array([[1, 2, 3], [4, 5, 6]]).shape)
print(np.array([[7, 8, 9]]).shape)

(2, 3)
(1, 3)


In [54]:
# find the most frames in an image and pad the others with zeros so that all have the same number of frames...
def max_frames(images):
    max_frames = 0
    for im in images:
        num_frames = im['X'].shape[2]
        if num_frames > max_frames:
            max_frames = num_frames
    return max_frames

def zero_padding(images, max_frams):
    new_ims = []
    for im in images:
        num_frames = im['X'].shape[2]
        if num_frames < max_frames:
            num_pad = max_frames-num_frames
            im_new = np.pad(im['X'], ((0, 0), (0, 0), (0, num_pad)), 'constant', constant_values = 0)
        new_ims.append(im_new)
    return new_ims


def zero_padding(images, max_frames):
    new_ims = []
    for im in images:
        num_frames = im['X'].shape[2]
        if num_frames < max_frames:
            num_pad = max_frames-num_frames
            im_new = np.pad(im['X'], ((0, 0), (0, 0), (0, num_pad)), 'constant', constant_values = 0)
        new_ims.append(im_new)
    return new_ims

def load_images(folder):
    images = [loadmat(os.path.join(folder, filename))['X'] if filename == *.mat for filename in os.listdir(folder)]
    classification = [loadmat(os.path.join(folder, filename))['TICI_report'] if filename == *.mat for filename in os.listdir(folder)]
    return images, classification
       
# now they are all the same shape
padded_ims = zero_padding(prelim_images)

In [ ]:
class ProcessImage:

    def __init__(self, folder=None):
        self.names = os.listdir(folder)
        self.folder = folder
      
    def __iter__(self):
        for filename in self.names:
            self.image = loadmat(os.path.join(self.folder, filename))
      


In [55]:
# use keras inception v3 to train on the time stamped image frames
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling3D

C:\Users\simir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def create_model():
    base_model = InceptionV3(weights='imagenet', include_top=False)

    # DNN portion
    # average pooling layer
    x = GlobalAveragePooling3D()(base_model.output)
    # fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # logistic layer
    predictions = Dense(5, activation='softmax')(x) # 5 is the number of classes in the data, since TICI is 0,1,2a,2b,3

    model = Model(inputs=base_model.input, outputs=predictions)

    #for layer in base_model.layers:
    #    layer.trainable = False

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def create_generators(x_train, y_train, batch_size):
    train_d_gen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True
        target_size = (256, 256))
    
    train_d_gen.fit(x_train)
    
    train_gen = train_d_gen.flow(x_train, y_train, batch_size)
    
    validation_gen = ImageDataGenerator().flow(x_test, y_test)
    
    return train_generator, validation_generator

def train_model(model, nb_epoch, generators, train_len, batch_size):
    train_generator, validation_generator = generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=train_len // batch_size,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=nb_epoch)
    return model

In [ ]:
# crossvalidation 
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import os

cv = StratifiedKFold(5)

tprs = []
aucs = []
i = 1

folder = ''

images, classification = load_images(folder)

model = create_model()
batch_size = 32

for train, test in cv.split(images, classification):
    x_train = images[train]
    y_train = classification[train]
    x_test = images[test]
    y_test = classification[test]
    
    generators = create_generators(x_train, y_train, batch_size)
    model = train_model(model, 10, generators, len(x_train), batch_size)
    
    Y_pred = model.predict(x_test)
    fpr, tprs, thresholds = roc_curve(y_test, Y_pred)
    tpr.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw = 1, alpha = 0.3, label = 'ROC fold %d (AUC - %0.2f)' % (i, roc_auc))
    plt.legend()
    i+=1
    
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Random Chance', alpha=.8)

mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc))

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('PLS-DA ROC Curve with Crossvalidation')
plt.legend(loc="lower right")
plt.savefig('ROC Curve for PLS-DA with Crossvalidation-14comp.jpg')